In [1]:
import numpy as np
import pandas as pd
import scipy.stats as st
from causallearn.search.ScoreBased.GES import ges
from causallearn.search.ConstraintBased.PC import pc
import itertools
from sklearn.preprocessing import StandardScaler
from dagma.nonlinear import DagmaMLP, DagmaNonlinear
from dagma.linear import DagmaLinear
from castle.algorithms import NotearsNonlinear, DAG_GNN
import os
os.environ["JAVA_HOME"] = "/homes/shahashka/lucid_cd/amazon-corretto-21.0.7.6.1-linux-x64/lib/"
import pytetrad.tools.TetradSearch as py_ts
import torch
import pickle
import networkx as nx
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

/homes/shahashka/.conda/envs/lucid_cd/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-05 12:12:58,751 - /homes/shahashka/.conda/envs/lucid_cd/lib/python3.10/site-packages/castle/backend/__init__.py[line:36] - INFO: You can use `os.environ['CASTLE_BACKEND'] = backend` to set the backend(`pytorch` or `mindspore`).
2025-05-05 12:12:58,900 - /homes/shahashka/.conda/envs/lucid_cd/lib/python3.10/site-packages/castle/algorithms/__init__.py[line:36] - INFO: You are using ``pytorch`` as the backend.


In [2]:
dose = "A"
df = pd.read_csv(f"data/huvec/cd_matrix_d{dose}.csv")
df_gene_expression = df.drop(columns=["radiation"])
num_genes = len(df_gene_expression.columns)
gene_names = df_gene_expression.columns
df_gene_expression = StandardScaler().fit_transform(df_gene_expression)
df_gene_expression = pd.DataFrame(data=df_gene_expression)
df_learn = pd.DataFrame(data=df_gene_expression)
df_learn.columns = gene_names
df_learn['radiation'] = df['radiation']
 

In [3]:
print(df)

    MYORG  WDR44  NSA2  MRPS26  PTGS2  IFT52  MAF  LOC105377627  DAPK3  MKNK2  \
0    1140   1618  3404    1157   2288   1550  128           190   5910   3456   
1    1402   2213  5139    1574   1413   2219  118            73   5513   8203   
2    1123   1785  4566    1190   1381   2008  138           252   5579   3954   
3    1126   1386  3888    1012    539   1954   70           172   5282   3925   
4     836   1487  2848     828   2276   1246   68           188   4821   2950   
5    1390   2009  4752    1514   1800   2316  218           379   6659   4280   
6    1262   1530  3964    1040    696   1934   88           201   5233   4082   
7     994   1532  4000    1282   1587   1819   28           134   4505   4895   
8    1110   1863  4550    1371   1411   2260   28            84   5282   5687   
9     966   1843  5222    1424   1148   2556   48           119   5360   6291   
10    865   1613  4360    1270    894   2186   54            96   4510   5327   
11   1024   1594  5256    13

In [4]:
#device = torch.device("mps" if torch.mps.is_available() else "cpu")
from jpype import JImplements, JOverride
import jpype.imports

import importlib.resources as importlib_resources
jar_path = importlib_resources.files('pytetrad').joinpath('resources','tetrad-current.jar')
jar_path = str(jar_path)
if not jpype.isJVMStarted():
    try:
        jpype.startJVM(jpype.getDefaultJVMPath(), classpath=[jar_path])
    except OSError:
        print("can't load jvm")
        pass

import pandas as pd

import pytetrad.tools.translate as tr

import java.util as util
import edu.cmu.tetrad.data as td
import edu.cmu.tetrad.search as ts
import edu.cmu.tetrad.search.score as sc

try:
    from causallearn.score.LocalScoreFunction import local_score_marginal_general
    from causallearn.score.LocalScoreFunction import local_score_cv_general
except ImportError as e:
    print('Could not import a causal-learn module: ', e)

# Can use this as a template for defining scores in Python for use with
# Java Tetrad algorithms.
@JImplements(sc.Score)
class Bgs:
    def __init__(self, df):
        self.df = df
        self.data = df.values
        self.parameters = {"kfold": 10, "lambda": 0.01}

        # pick a score: bug in marginal_general?
        # self.score = local_score_marginal_general
        self.score = local_score_cv_general

        # these scores are expensive, so caching seems pertinent...
        self.cache = {}

        self.variables = util.ArrayList()
        self.variable_map = {}
        for col in df.columns:
            col = str(col)
            variable = td.ContinuousVariable(col)
            self.variables.add(variable)
            self.variable_map[col] = variable

    # camelCase is java convention; mathcing that...
    def setParameters(self, parameters):
        self.paramaters = parameters

    @JOverride
    def localScore(self, *args):
        Xi = args[0] 
        if len(args) == 1: PAi = []
        elif isinstance(args[1], int): PAi = [args[1]]
        else: PAi = list(args[1])

        key = (Xi, *sorted(PAi))
        if key not in self.cache:
            self.cache[key] = self.score(self.data, Xi, PAi, self.parameters)
 
        # for debugging...
        # print(key, self.cache[key])
        return self.cache[key]

    @JOverride
    def localScoreDiff(self, *args):
        Xi = args[0]
        if len(args) == 2: PAi = []
        else: PAi = list(args[2])

        diff = -self.localScore(Xi, PAi)
        PAi.append(args[1])
        diff += self.localScore(Xi, PAi)

        return diff 
    
    @JOverride
    def getVariables(self):
        return self.variables
    
    @JOverride
    def getSampleSize(self):
        return self.n

    @JOverride
    def toString(self):
        return "Biwei's General Score"

    @JOverride
    def getVariable(self, targetName):
        if targetName in self.variable_map: 
            return self.variable_map[targetName]
        return None

    @JOverride
    def isEffectEdge(self, bump):
        return False

    @JOverride
    def getMaxDegree(self):
        return 1000

    @JOverride
    def defaultScore(self):
        return self
def dagma_nonlinear_local_learn(subproblem, use_skel):
    skel, data = subproblem
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    d = data.shape[1]
    X = data.values
    X_tensor = torch.from_numpy(X).type(torch.double).to(device)
    eq_model = DagmaMLP(dims=[d, np.min([10, d-1]), 1], bias=True, dtype=torch.double) # create the model for the structural equations, in this case MLPs
    eq_model = eq_model.to(device)
    model = DagmaNonlinear(eq_model, dtype=torch.double) # create the model for DAG learning
    # adj = model.fit(X_tensor, lambda1=0.02, lambda2=0.005) # fit the model with L1 reg. (coeff. 0.02) and L2 reg. (coeff. 0.005)
    adj = model.fit(X_tensor, lambda1=0.001, lambda2=0.0001) # fit the model with L1 reg. (coeff. 0.02) and L2 reg. (coeff. 0.005)

    return adj

def ges_non_param_local_learn(subproblem, use_skel):
    skel, data = subproblem
    result = ges(data.values, maxP=10, node_names=data.columns, score_func="local_score_CV_general")
    return result
def ges_discrete_poisson_local_learn(subproblem, use_skel):
    skel, data = subproblem
    result = ges(data.values, maxP=10, node_names=data.columns, score_func="local_score_BDeu")
    return result
def pc_kci_local_learn(subproblem, use_skel):
    skel, data = subproblem
    result = pc(data.values, alpha=0.05, indep_test='kci')
    return result
def dag_gnn_local_learn(subproblem, use_skel):
    skel, data = subproblem
    d = data.shape[1]
    X = data.values
    model = DAG_GNN(device_type="gpu")
    model.learn(X)
    return model.causal_matrix
    
def notears_mlp_local_learn(subproblem, use_skel):
    skel, data = subproblem
    d = data.shape[1]
    X = data.values
    model = NotearsNonlinear(device_type="gpu")
    adj = model.learn(X)
    return adj

def pc_kci_tetrad_local_learn(subproblem, use_skel):
    skel, data = subproblem
    data = data.astype({col: "float64" for col in data.columns})
    search = py_ts.TetradSearch(data)
    search.set_verbose(False)
    search.use_kci(alpha=0.1)
    #search.use_fisher_z(alpha=0.05)
    ## Run various algorithms and print their results. For now (for compability with R)
    search.run_pc()
    adj = search.get_graph_to_matrix().values
    return adj

def ges_non_param_tetrad_local_learn(subproblem, use_skel):
    skel, data = subproblem
    data = data.astype({col: "float64" for col in data.columns})
    # Java calls
    score = Bgs(data)
    graph = ts.Fges(score).search()
    # convert to python
    adj = tr.graph_to_matrix(graph).values
    return adj

def ges_bic_tetrad_local_learn(subproblem, use_skel):
    skel, data = subproblem
    data = data.astype({col: "float64" for col in data.columns})
    search = py_ts.TetradSearch(data)
    search.set_verbose(False)
    search.use_sem_bic()
    #search.use_fisher_z(alpha=0.05)
    ## Run various algorithms and print their results. For now (for compability with R)
    search.run_fges()
    adj = search.get_graph_to_matrix().values
    return adj

### Global learning 

In [5]:
ground_truth = nx.read_gexf(f"./data/huvec/cd_subgraph_d{dose}.gexf" )
print(ground_truth)

DiGraph with 918 nodes and 1670 edges


In [6]:
from src.GENIE3 import GENIE3
# global_adj = ges_local_learn([superstructure, df], use_skel=True)
#global_adj = dagma_nonlinear_local_learn([None, df_learn], use_skel=False)
#global_adj = dag_gnn_local_learn([None, df[list(ground_truth.nodes)+['radiation']]], use_skel=False)
#global_adj = notears_mlp_local_learn([None, df.iloc[:,0:1000]], use_skel=False)
#print(np.sum(global_adj!=0))
global_adj = GENIE3(df[list(ground_truth.nodes)+['radiation']].values, nthreads=16)


Tree method: RF
K: sqrt
Number of trees: 1000


running jobs on 16 threads
Elapsed time: 59.13 seconds


In [7]:
import networkx as nx
global_net = nx.from_numpy_array(global_adj, create_using=nx.DiGraph)
global_net = nx.relabel_nodes(global_net, dict(zip(np.arange(len(global_net.nodes)), list(ground_truth.nodes)+['radiation'])))
print(global_net)

DiGraph with 919 nodes and 842076 edges


In [ ]:
true_positive = 0
for edge in ground_truth.edges:
    if global_net.has_edge(edge[0], edge[1]):
        true_positive += 1
        print(edge)
print(true_positive)
print(len(list(global_net.neighbors("radiation"))))
sorted_nodes = sorted(global_net.out_degree(), key=lambda x: x[1], reverse=True)

# Print nodes and their degree
print("Nodes sorted by degree:")
for node, degree in sorted_nodes:
    print(f"{node}: degree {degree}")

('GADD45A', 'EEF1A1')
1
245
Nodes sorted by degree:
radiation: degree 245
COL1A1: degree 18
BCL2A1: degree 11
NUPR1: degree 9
FOSB: degree 8
CDKN2A: degree 7
ADRA1D: degree 7
ADIRF-AS1: degree 6
GLI2: degree 5
DSC3: degree 4
SIX2: degree 4
CCNA1: degree 4
ACE: degree 3
FLRT3: degree 3
ESR2: degree 3
AMPH: degree 3
GRB14: degree 3
MYCN: degree 3
NTRK2: degree 3
ANGPTL4: degree 3
MAF: degree 3
ASS1: degree 3
C1QTNF6: degree 3
HMOX1: degree 3
BST2: degree 2
TUBA1A: degree 2
EFNB1: degree 2
CCND2: degree 2
SELL: degree 2
SMAD7: degree 2
TAGLN2: degree 2
HSPA2: degree 2
HDAC9: degree 2
ICAM1: degree 2
MT2A: degree 2
TUBB3: degree 2
CENPV: degree 2
BAG2: degree 2
DKK1: degree 2
MME: degree 2
ZSCAN20: degree 2
PCDH12: degree 2
GPR4: degree 2
TUBA4A: degree 2
TUBB2B: degree 2
PDLIM7: degree 2
CDKN1A: degree 1
GADD45A: degree 1
SGK1: degree 1
GYPC: degree 1
PROX1: degree 1
CDC6: degree 1
NOTCH3: degree 1
HERC5: degree 1
FBXO32: degree 1
CENPF: degree 1
RPS27A: degree 1
NTN4: degree 1
PIF1: degr

### Superstructure

In [19]:

# Correlation matrix with Permutation testing
corr_mat = df_gene_expression.corr('pearson', numeric_only=True).to_numpy()
print(np.min(np.abs(corr_mat)))
np.fill_diagonal(corr_mat, 0)
print(np.max(corr_mat), np.argmax(corr_mat))
random_corr_coef = []
for i in range(10):
    shuffled_array = df_gene_expression.values
    [np.random.shuffle(x) for x in shuffled_array]
    shuffled_final_data_set = pd.DataFrame(data=shuffled_array)
    shuffle_corr_mat = shuffled_final_data_set.corr('pearson', numeric_only=True).to_numpy()
    np.fill_diagonal(shuffle_corr_mat, 0)
    print(np.max(shuffle_corr_mat), np.argmax(shuffle_corr_mat))
    random_corr_coef.append(np.max(shuffle_corr_mat))
print(random_corr_coef)
ci_interval = st.t.interval(0.95, len(random_corr_coef)-1, 
                            loc=np.mean(random_corr_coef), 
                            scale=st.sem(random_corr_coef))
print(ci_interval)
cutoff = ci_interval[1]

corr_mat[corr_mat<=cutoff] = 0
corr_mat[corr_mat>cutoff] = 1

superstructure = corr_mat
print(f"Superstructure contains {np.sum(superstructure)} edges which is \
        {np.sum(superstructure)/(superstructure.shape[1]**2)} fraction of all possible edges")

nan
nan 5886
0.9931338554691231 2400661
0.9922982607314539 19666634
0.9885011722553623 18559022
0.9933796287091708 5287612
0.9899321810732494 3587886
0.9894394392309578 9951651


KeyboardInterrupt: 

In [9]:
# STRING PPI
ppi_df = pd.read_csv("./data/prior_knowledge/string_ppi_gene_names.csv", header=0)
string_undirected_edges = {(start, end) for start, end in zip(ppi_df["protein1"], ppi_df["protein2"])}
ppi_superstructure = np.zeros((num_genes, num_genes))
for i, j in itertools.product(np.arange(num_genes), np.arange(num_genes)):
    start = df.columns[i]
    end = df.columns[j]
    if (start, end) in string_undirected_edges:
        ppi_superstructure[i][j] = 1
        
print(f"Superstructure contains {np.sum(ppi_superstructure)} edges which is \
        {np.sum(ppi_superstructure)/(ppi_superstructure.shape[1]**2)} fraction of all possible edges")



Superstructure contains 298183.0 edges which is         0.008606812943491176 fraction of all possible edges


In [8]:
superstructure = ppi_superstructure

### Partition network

In [6]:
with open(f"./data/huvec/cd_partition_d{dose}_new.pickle", 'rb') as f:
    custom_partition = pickle.load(f)
print(custom_partition)
for i, comm in custom_partition.items():
    comm += ['radiation']

{0: ['TAPBPL', 'PDIA3', 'DKK3', 'ETS1', 'FOXP1', 'FLI1', 'SHC1', 'CBL', 'VAV2', 'CTSD', 'ACE', 'CXCR4', 'JUND', 'CD9', 'VAV3', 'E2F1', 'B2M', 'UBC', 'NFIC', 'ATF2', 'CDR1', 'ETS2', 'RANGAP1', 'ELF1', 'LIN54', 'BCOR', 'PLCG2', 'MICA', 'GADD45A', 'VCP', 'FYN', 'PIK3CB', 'HMBS', 'SEC24D', 'CD82', 'WDR5', 'SLC4A2', 'BRD4', 'STAT2', 'MED12', 'KLF4', 'LNPEP', 'HFE', 'LMNB1', 'ATP8A2', 'ADRA1D', 'ATP8A1', 'CANX', 'DLGAP5', 'KPNA2', 'UBB', 'IRF1', 'ERG', 'MAP1S', 'HDAC1', 'BHLHE40', 'SLC4A11', 'STAT5A', 'CALR', 'RAD21', 'ARG2', 'FCGRT', 'GYPC', 'MAF', 'MYC', 'NOTCH1', 'SLC4A8', 'MAZ', 'CDKN1A', 'IRF2', 'PTPRJ', 'PIK3R1', 'TPST1', 'PCNA', 'TFRC', 'HLA-A', 'NUSAP1', 'HNRNPC', 'SEC23A', 'AR', 'ICAM1', 'EGR1', 'ZNF263', 'PROCR'], 1: ['BAG2', 'ETS1', 'REPIN1', 'AGPAT2', 'ATAD3B', 'AXIN2', 'RCC1', 'UTP14A', 'RHOT1', 'PALD1', 'IGF1R', 'JUN', 'UBC', 'STK40', 'UBE2C', 'GPR4', 'TSPAN13', 'WRN', 'SREBF2', 'CLIP1', 'BCOR', 'PA2G4', 'RFC3', 'KIF23', 'N4BP3', 'SRSF3', 'FLCN', 'SSR2', 'RFC4', 'PLSCR1', 'EIF3

In [7]:
def partition_problem(partition: dict, structure: np.ndarray, data: pd.DataFrame):
    sub_problems = []
    k = list(partition.keys())
    k.sort()
    for i in k:
        sub_nodes = partition[i]
        sub_structure = np.ones((len(sub_nodes), len(sub_nodes)))
        sub_nodes = list(sub_nodes)
        sub_data = data[sub_nodes] 
        sub_problems.append((sub_structure, sub_data))
    return sub_problems

### Learning subgraphs

In [8]:
import functools
from concurrent.futures import ProcessPoolExecutor

superstructure = np.ones((num_genes+1, num_genes+1))
subproblems = partition_problem(custom_partition, superstructure, df)
# Setup thread level parallelism
# func_partial = functools.partial(
#     dagma_nonlinear_local_learn,
#     use_skel=False,
# )
# n_comms = len(ec_partition_small)
# nthreads = n_comms  # each thread handles one partition
results = []
# chunksize = max(1, n_comms // nthreads)
for subproblem in subproblems:
    print(subproblem[1].columns)
    #adj = notears_mlp_local_learn(subproblem, use_skel=False)
    #adj = ges_discrete_poisson_local_learn(subproblem, use_skel=False)
    adj = ges_non_param_tetrad_local_learn(subproblem, use_skel=False)
    print(type(adj))
    #adj = ges_bic_tetrad_local_learn(subproblem, use_skel=False)
    print(np.sum(adj!=0))
    results.append(adj)
# with ProcessPoolExecutor(max_workers=nthreads) as executor:
#     for result in executor.map(func_partial, subproblems, chunksize=chunksize):
#         results.append(result)

Index(['TAPBPL', 'PDIA3', 'DKK3', 'ETS1', 'FOXP1', 'FLI1', 'SHC1', 'CBL',
       'VAV2', 'CTSD', 'ACE', 'CXCR4', 'JUND', 'CD9', 'VAV3', 'E2F1', 'B2M',
       'UBC', 'NFIC', 'ATF2', 'CDR1', 'ETS2', 'RANGAP1', 'ELF1', 'LIN54',
       'BCOR', 'PLCG2', 'MICA', 'GADD45A', 'VCP', 'FYN', 'PIK3CB', 'HMBS',
       'SEC24D', 'CD82', 'WDR5', 'SLC4A2', 'BRD4', 'STAT2', 'MED12', 'KLF4',
       'LNPEP', 'HFE', 'LMNB1', 'ATP8A2', 'ADRA1D', 'ATP8A1', 'CANX', 'DLGAP5',
       'KPNA2', 'UBB', 'IRF1', 'ERG', 'MAP1S', 'HDAC1', 'BHLHE40', 'SLC4A11',
       'STAT5A', 'CALR', 'RAD21', 'ARG2', 'FCGRT', 'GYPC', 'MAF', 'MYC',
       'NOTCH1', 'SLC4A8', 'MAZ', 'CDKN1A', 'IRF2', 'PTPRJ', 'PIK3R1', 'TPST1',
       'PCNA', 'TFRC', 'HLA-A', 'NUSAP1', 'HNRNPC', 'SEC23A', 'AR', 'ICAM1',
       'EGR1', 'ZNF263', 'PROCR', 'radiation'],
      dtype='object')


Apr 28, 2025 1:48:39 PM java.util.prefs.FileSystemPreferences$6 run


<class 'numpy.ndarray'>
0
Index(['BAG2', 'ETS1', 'REPIN1', 'AGPAT2', 'ATAD3B', 'AXIN2', 'RCC1', 'UTP14A',
       'RHOT1', 'PALD1',
       ...
       'RNASEH1', 'HLA-A', 'KAT7', 'RPL23', 'TJP2', 'MAFK', 'IFI16', 'CDC25C',
       'TXNDC12', 'radiation'],
      dtype='object', length=786)
<class 'numpy.ndarray'>
0
Index(['NCL', 'ERMAP', 'ETS1', 'FOXP1', 'REPIN1', 'ABL1', 'CCNG2', 'SNTB2',
       'CDK16', 'CHEK1',
       ...
       'LATS2', 'MYCN', 'NCOR2', 'UBA52', 'BRCA1', 'ZNF263', 'FLI1', 'MAFK',
       'FOXO1', 'radiation'],
      dtype='object', length=185)
<class 'numpy.ndarray'>
0
Index(['ETS1', 'FOXP1', 'REPIN1', 'EEF1A1', 'JUND', 'GADD45B', 'MAP2K6',
       'E2F1', 'B2M', 'MBD4', 'JUN', 'NFIC', 'FOSL2', 'BCOR', 'NRIP1', 'NPM1',
       'RXRA', 'GADD45A', 'GMNN', 'BCL2L11', 'LMO2', 'GATA2', 'MAP3K4', 'BRD4',
       'MED12', 'KLF4', 'VDR', 'MEF2A', 'LMNB1', 'DCTN2', 'GATA3', 'MECOM',
       'KMT2B', 'SMARCA4', 'JUNB', 'ATM', 'EWSR1', 'IRF1', 'SMAD6', 'ERG',
       'ESRRA', 'CDK1', '

In [9]:
from cd_v_partition.fusion import screen_projections, fusion
num_edges = 0
for r in results:
    print(r.shape)
    num_edges += np.sum(r != 0)
print(num_edges)
global_net_non_param = screen_projections(superstructure, custom_partition, results, ss_subset=False, finite_lim=False, data=df_learn.to_numpy())
print(len(global_net_non_param.edges()))
print(len(global_net_non_param.nodes()))

(85, 85)
(786, 786)
(185, 185)
(71, 71)
0
0
919


In [ ]:
import networkx as nx
from netgraph import Graph
import matplotlib.pyplot as plt
ground_truth = nx.read_gexf(f"./data/huvec/cd_subgraph_d{dose}.gexf" )
print(ground_truth)
true_positive = 0
for edge in ground_truth.edges:
    if  global_net_non_param.has_edge(edge[0], edge[1]):
        true_positive += 1
        print(edge)
print(true_positive)

DiGraph with 1487 nodes and 3384 edges
('EGR2', 'POLR2L')
('EGR2', 'TEAD1')
('EGR2', 'POLR2B')
('EGR2', 'POLR2D')
('EGR2', 'WWTR1')
('EGR2', 'SCD5')
('EGR2', 'TTC3')
('EGR2', 'MICU1')
('EGR2', 'TMEM87B')
('EGR2', 'NUP188')
('EGR2', 'RBM39')
('EGR2', 'DONSON')
('EGR2', 'SMPD1')
('EGR2', 'FAF1')
('EGR2', 'ATP6V1G1')
('EGR2', 'LTBP3')
('EGR2', 'NPLOC4')
('EGR2', 'CCNI')
('EGR2', 'TMX4')
('EGR2', 'SLFN11')
('EGR2', 'SH3TC1')
('EGR2', 'BZW2')
('EGR2', 'PNRC1')
('EGR2', 'CAPN1')
('VAV3', 'B2M')
25


In [52]:
radbio_df = pd.read_csv("./data/prior_knowledge/radiation_edges.txt")
radbio_net = nx.DiGraph({edge for edge in zip(radbio_df['node1'], radbio_df['node2'])})
true_positive=0
for edge in radbio_net.edges:
    if  global_net_non_param.has_edge(edge[0], edge[1]):
        true_positive += 1
        print(edge)
print(true_positive)

0


In [53]:
nx.write_gexf(global_net_non_param, f"./data/huvec/cd_dag_gnn_d{dose}.gexf")

In [54]:
len(list(global_net_non_param.neighbors("radiation")))

135

In [55]:
sorted_nodes = sorted(global_net_non_param.out_degree(), key=lambda x: x[1], reverse=True)

# Print nodes and their degree
print("Nodes sorted by degree:")
for node, degree in sorted_nodes:
    print(f"{node}: degree {degree}")

Nodes sorted by degree:
FOXS1: degree 176
radiation: degree 135
EGR2: degree 129
CRYAB: degree 88
ADRA1D: degree 45
BCL2A1: degree 35
COL1A1: degree 33
CD38: degree 31
KLF4: degree 30
NUPR1: degree 28
MAFB: degree 26
IGF2: degree 25
ATP8A2: degree 23
CEP19: degree 21
SMAD6: degree 19
HFE: degree 16
MAF: degree 15
ICAM1: degree 15
FOSB: degree 14
GYPC: degree 13
WDR27: degree 13
KCNMA1: degree 12
VAV3: degree 12
TPST1: degree 12
PTPN7: degree 12
PPP1R14A: degree 10
GLI2: degree 10
BDKRB2: degree 10
SMAD7: degree 9
ACE: degree 9
TMEM184A: degree 9
BCL2L11: degree 9
NFATC2: degree 9
PABPC1L: degree 8
MAP3K14: degree 8
LRRC27: degree 8
TXNIP: degree 7
TRPC6: degree 7
CCNA1: degree 7
ATP8A1: degree 7
PTPN22: degree 6
EGR1: degree 6
PTGS2: degree 6
PBX3: degree 6
SLC4A11: degree 6
PTPRJ: degree 6
CCND2: degree 6
CDKN2C: degree 6
SETBP1: degree 5
VCAN: degree 5
ATF3: degree 5
LIN54: degree 5
SLC4A8: degree 5
ZNF263: degree 5
SIX2: degree 5
STAG3: degree 5
GADD45B: degree 5
LMO2: degree 5
GATA

# BEELINE


In [10]:
df_bl = pd.read_csv('./data/BEELINE/BEELINE-data/inputs/scRNA-Seq/hESC/ExpressionData.csv')
df_bl_net = pd.read_csv('./data/BEELINE/Networks/human/STRING-network.csv')
print(df_bl)
print(df_bl_net)

      Unnamed: 0  H9_00hb4s_001  H9_00hb4s_002  H9_00hb4s_003  H9_00hb4s_004  \
0           A1BG       2.510133       0.000000       1.702251       4.259372   
1           A1CF       1.424896       1.208699       0.000000       0.000000   
2          A2LD1       0.000000       0.000000       0.000000       0.000000   
3            A2M       0.000000       0.000000       0.000000       0.000000   
4          A2ML1       2.141105       2.936250       0.220701       0.776599   
...          ...            ...            ...            ...            ...   
17730     ZYG11A       2.389235       3.615408       2.744771       3.306575   
17731     ZYG11B       7.528404       3.703729       6.631982       6.525436   
17732        ZYX       4.713966       5.265492       1.323498       4.049480   
17733      ZZEF1       0.000000       3.429429       2.461572       1.199451   
17734       ZZZ3       8.038216       6.994479       6.360136       5.826560   

       H9_00hb4s_005  H9_00hb4s_006  H9

In [21]:
df_bl_GENELink_500 = pd.read_csv('./src/GENELink/Code/Demo/hESC/TFs+500/BL--ExpressionData.csv')
df_bl_GENELink_500_net = pd.read_csv('./src/GENELink/Code/Demo/hESC/TFs+500/BL--network.csv')
print(df_bl_GENELink_500)
print(df_bl_GENELink_500_net)
num_samples = df_bl_GENELink_500.shape[1] - 1
num_genes = df_bl_GENELink_500.shape[0]

    Unnamed: 0  H9_00hb4s_001  H9_00hb4s_002  H9_00hb4s_003  H9_00hb4s_004  \
0       KIF21A       7.679538       7.914004       7.409868       6.226928   
1         DPH2       5.312810       5.041560       4.002452       6.894618   
2          FUS       5.281794       5.503217       5.938976       5.313464   
3       NKX3-1       0.000000       5.974976       0.000000       0.000000   
4       TRIM33       7.996107       5.104049       5.723665       1.845207   
..         ...            ...            ...            ...            ...   
905        GCA       5.415605       6.825231       7.441517       6.846669   
906      NOL11       8.318358       9.662753       6.790495       9.021283   
907      KRCC1       6.158881       2.744434       4.730438       0.000000   
908      TEAD4       4.951478       3.810594       2.249993       3.507437   
909       OXTR       4.480848       3.443538       1.969116       3.009541   

     H9_00hb4s_005  H9_00hb4s_006  H9_00hb4s_007  H9_00hb4s_008

In [14]:
df_bl_GENELink_1000 = pd.read_csv('./src/GENELink/Code/Demo/hESC/TFs+1000/BL--ExpressionData.csv')
df_bl_GENELink_1000_net = pd.read_csv('./src/GENELink/Code/Demo/hESC/TFs+1000/BL--network.csv')

print(df_bl_GENELink_1000)
print(df_bl_GENELink_1000_net)


     Unnamed: 0  H9_00hb4s_001  H9_00hb4s_002  H9_00hb4s_003  H9_00hb4s_004  \
0         PDCD2       4.023643       4.714161       4.885170       4.526124   
1         MBTD1       5.116579       5.542572       4.360040       4.158240   
2         SMAD2       9.309451       9.288134       8.959270       8.607650   
3        ARID3B       6.294547       6.488272       3.783785       7.099452   
4        HOMER1       6.720808       5.898247       2.001962       6.519740   
...         ...            ...            ...            ...            ...   
1405    LEPREL1       5.361625       0.000000       3.428637       0.000000   
1406      CAPN2       2.510133       3.838789       5.657773       1.493202   
1407      WNT5A       6.532634       2.090131       2.249993       0.000000   
1408      USP44       7.754054       9.315373       8.560073       8.186961   
1409      CEP55       3.377603       5.753293       5.763919       2.903542   

      H9_00hb4s_005  H9_00hb4s_006  H9_00hb4s_007  

In [ ]:
# Run CD
data = df_bl_GENELink_500.iloc[:,1:num_samples+1].T.values
df_bl_GENELink_500_learn = pd.DataFrame(data=data)
df_bl_GENELink_500_learn.columns = df_bl_GENELink_500.iloc[:,0]
print(df_bl_GENELink_500_learn)
global_adj_bl = dag_gnn_local_learn([None, df_bl_GENELink_500_learn], use_skel=False)

2025-04-29 10:07:07,586 - /homes/shahashka/.conda/envs/lucid_cd/lib/python3.10/site-packages/castle/algorithms/gradient/dag_gnn/torch/dag_gnn.py[line:165] - INFO: GPU is available.


Unnamed: 0    KIF21A      DPH2       FUS    NKX3-1    TRIM33  HOXB4      NONO  \
0           7.679538  5.312810  5.281794  0.000000  7.996107    0.0  9.660731   
1           7.914004  5.041560  5.503217  5.974976  5.104049    0.0  9.657859   
2           7.409868  4.002452  5.938976  0.000000  5.723665    0.0  8.919415   
3           6.226928  6.894618  5.313464  0.000000  1.845207    0.0  9.211901   
4           5.782105  4.296249  5.338775  0.000000  5.295540    0.0  9.602588   
..               ...       ...       ...       ...       ...    ...       ...   
753         4.499911  6.451278  3.817104  0.000000  6.569268    0.0  8.369346   
754         6.306732  0.000000  2.891619  0.000000  0.000000    0.0  8.142321   
755         0.000000  0.693009  4.213365  0.000000  5.098737    0.0  8.435509   
756         0.000000  1.510372  7.180517  0.000000  6.263076    0.0  9.174655   
757         5.174988  3.759710  5.108344  0.000000  3.350471    0.0  8.494338   

Unnamed: 0    PHLDB2   SMAR

2025-04-29 10:08:05,759 - /homes/shahashka/.conda/envs/lucid_cd/lib/python3.10/site-packages/castle/algorithms/gradient/dag_gnn/torch/dag_gnn.py[line:253] - INFO: Iter: 0, epoch: 299, h_new: 1.0
2025-04-29 10:08:59,835 - /homes/shahashka/.conda/envs/lucid_cd/lib/python3.10/site-packages/castle/algorithms/gradient/dag_gnn/torch/dag_gnn.py[line:253] - INFO: Iter: 1, epoch: 299, h_new: 1.0
2025-04-29 10:09:53,986 - /homes/shahashka/.conda/envs/lucid_cd/lib/python3.10/site-packages/castle/algorithms/gradient/dag_gnn/torch/dag_gnn.py[line:253] - INFO: Iter: 2, epoch: 299, h_new: 1.0
2025-04-29 10:10:48,092 - /homes/shahashka/.conda/envs/lucid_cd/lib/python3.10/site-packages/castle/algorithms/gradient/dag_gnn/torch/dag_gnn.py[line:253] - INFO: Iter: 3, epoch: 299, h_new: 1.0
2025-04-29 10:11:42,238 - /homes/shahashka/.conda/envs/lucid_cd/lib/python3.10/site-packages/castle/algorithms/gradient/dag_gnn/torch/dag_gnn.py[line:253] - INFO: Iter: 4, epoch: 299, h_new: 1.0
2025-04-29 10:12:36,297 -

In [ ]:
# Evaluation
ground_truth_net = nx.from_edgelist(df_bl_GENELink_500_net.values.tolist(), create_using=nx.DiGraph)
print(ground_truth_net)
sorted_nodes = sorted(ground_truth_net.out_degree(), key=lambda x: x[1], reverse=True)
# Print nodes and their degree
print("Ground truth graph nodes sorted by degree:")
for node, degree in sorted_nodes:
    print(f"{node}: degree {degree}")
    
global_net_bl = nx.from_numpy_array(global_adj_bl, create_using=nx.DiGraph)
global_net_bl = nx.relabel_nodes(global_net_bl, dict(zip(np.arange(len(global_net.nodes)), list(df_bl_GENELink_500_learn.columns))))
print(global_net_bl)
true_positive = 0
for edge in ground_truth_net.edges:
    if global_net.has_edge(edge[0], edge[1]):
        global_net_bl += 1
        print(edge)
print(f"Number of true positive edges {true_positive}")
sorted_nodes = sorted(global_net_bl.out_degree(), key=lambda x: x[1], reverse=True)
# Print nodes and their degree
print("Estimated graph nodes sorted by degree:")
for node, degree in sorted_nodes:
    print(f"{node}: degree {degree}")


In [1]:
from src.GENELink.Code.Train_Test_Split import Hard_Negative_Specific_train_test_val
path = "./data/huvec/GENELink_data_files"
for dose in ["A", "B", "C", "D", "E"]:
    Hard_Negative_Specific_train_test_val(f"{path}/Label_{dose}.csv", f"{path}/Target_{dose}.csv", 
                                          f"{path}/TF_{dose}.csv", f"{path}/Train_set_{dose}.csv", 
                                          f"{path}/Validation_set_{dose}.csv", f"{path}/Test_set_{dose}.csv",
                                          ratio=0.67, p_val=0.5)